In [5]:
import os
import base64
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time
import re

# OpenAI API Key (replace with your own API key)
my_api_key = "your_openai_api_key"
client = OpenAI(api_key=my_api_key, timeout=20.0)

class DialogModel:
    def __init__(self, engine="gpt-4o"):
        """
        Initialize the DialogModel class.
        :param engine: Name of the GPT engine to use
        """
        self.engine = engine
        self.messages = []

    def set_prompts(self, system_prompt, user_prompt):
        """
        Set the system prompt and user prompt and initialize the message array.
        :param system_prompt: Common system prompt
        :param user_prompt: User prompt specific to the model
        """
        self.messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
    def send_message(self, message=None):
        """
        Send a message to the model and receive the response.
        :param message: Message to send
        :return: Model response
        """
        if message:
            self.messages.append({"role": "user", "content": message})
        
        for attempt in range(5):  # Maximum 5 attempts
            try:
                response = client.chat.completions.create(
                    model=self.engine,
                    messages=self.messages,
                    max_tokens=300,
                    temperature=1,
                )
                break
            except Exception as e:
                print(f"An error occurred during send_message attempt {attempt + 1}: {e}")
                if 'Connection error' in str(e):
                    print("Connection error detected. Waiting for 5 minutes before retrying...")
                    time.sleep(300)  # Wait for 5 minutes
                else:
                    time.sleep(1)  # Wait for 1 second for other errors
                if attempt == 4:
                    print(f"Failed to process message after 5 attempts. Skipping file.")
                    return None

        reply = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": reply})
        return reply

    def reset_dialog(self):
        """
        Reset the dialog history.
        """
        self.messages = []
        
    def update_message_with_reply(self, reply):
        """
        Add the other model's response to the current model's message array.
        :param reply: Response message to add
        """
        self.messages.append({"role": "user", "content": reply})
    
    def add_round_marker(self, round_number):
        """
        Add a round marker to the conversation.
        :param round_number: Round number
        """
        round_marker = f"===round{round_number}==="
        self.messages.append({"role": "user", "content": round_marker})

def extract_proposal(dialogue):
    for attempt in range(5):  # Maximum 5 attempts
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Based on the following dialogue, extract the amount the proposer proposed for him or her self against the responder. Answer with only one number. Do not use '$'."},
                    {"role": "user", "content": f"Dialogue:\n{dialogue}"}
                ],
                temperature=0.0,
                max_tokens=150
            )
            break
        except Exception as e:
            print(f"An error occurred during extract_proposal attempt {attempt + 1}: {e}")
            if 'Connection error' in str(e):
                print("Connection error detected. Waiting for 5 minutes before retrying...")
                time.sleep(300)  # Wait for 5 minutes
            else:
                time.sleep(1)  # Wait for 1 second for other errors
            if attempt == 4:
                print(f"Failed to extract proposal after 5 attempts. Skipping this step.")
                return None

    return response.choices[0].message.content.strip()

def extract_acceptance(dialogue):
    for attempt in range(5):  # Maximum 5 attempts
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Based on the following dialogue, If the responder accepts the proposal, print '1', and if it rejects, print '0'. Answer with only one number."},
                    {"role": "user", "content": f"Dialogue:\n{dialogue}"}
                ],
                temperature=0.0,
                max_tokens=150
            )
            break
        except Exception as e:
            print(f"An error occurred during extract_acceptance attempt {attempt + 1}: {e}")
            if 'Connection error' in str(e):
                print("Connection error detected. Waiting for 5 minutes before retrying...")
                time.sleep(300)  # Wait for 5 minutes
            else:
                time.sleep(1)  # Wait for 1 second for other errors
            if attempt == 4:
                print(f"Failed to extract acceptance after 5 attempts. Skipping this step.")
                return None

    return response.choices[0].message.content.strip()

def automated_dialogue(model1, model2, number_of_turns):
    """
    Conducts a conversation between Model 1 and Model 2.
    :param model1: First model to generate dialogue
    :param model2: Second model to generate dialogue
    :param number_of_turns: Number of dialogue turns
    """
    dialogue_results = ""
    rounds_data = []
    dialogue_data = []

    for turn in range(number_of_turns):
        round_info = {'round': turn + 1, 'model1_role': '', 'model1_text': '', 'proposal_amount': '',
                      'model2_text': '', 'proposal_acceptance': ''}

        if turn % 2 == 0:
            model1.add_round_marker(turn + 1)
            reply_from_model1 = model1.send_message()
            if reply_from_model1 is None:
                continue
            proposal_amount = extract_proposal(reply_from_model1)
            round_info['model1_role'] = 'Proposer'
            round_info['model1_text'] = reply_from_model1
            round_info['proposal_amount'] = proposal_amount
            dialogue_results += f"Round {turn+1}, Model 1 (Proposer): {reply_from_model1}\n"
            model2.update_message_with_reply(reply_from_model1)

            reply_from_model2 = model2.send_message()
            if reply_from_model2 is None:
                continue

            proposal_acceptance = extract_acceptance(reply_from_model2)
            round_info['model2_text'] = reply_from_model2
            round_info['proposal_acceptance'] = proposal_acceptance
            dialogue_results += f"Round {turn+1}, Model 2 (Responder): {reply_from_model2}\n"
            model1.update_message_with_reply(reply_from_model2)

            dialogue_data.extend([round_info['model1_text'], round_info['model2_text'], round_info['proposal_amount'], round_info['proposal_acceptance']])

        else:
            model2.add_round_marker(turn + 1)
            reply_from_model2 = model2.send_message()
            if reply_from_model2 is None:
                continue
            
            proposal_amount = extract_proposal(reply_from_model2)
            round_info['model2_role'] = 'Proposer'
            round_info['model2_text'] = reply_from_model2
            round_info['proposal_amount'] = proposal_amount
            dialogue_results += f"Round {turn+1}, Model 2 (Proposer): {reply_from_model2}\n"
            model1.update_message_with_reply(reply_from_model2)

            reply_from_model1 = model1.send_message()
            if reply_from_model1 is None:
                continue
            
            proposal_acceptance = extract_acceptance(reply_from_model1)
            round_info['model1_text'] = reply_from_model1
            round_info['proposal_acceptance'] = proposal_acceptance
            dialogue_results += f"Round {turn+1}, Model 1 (Responder): {reply_from_model1}\n"
            model2.update_message_with_reply(reply_from_model1)

            dialogue_data.extend([round_info['model2_text'], round_info['model1_text'], round_info['proposal_amount'], round_info['proposal_acceptance']])

        rounds_data.append(round_info)

    columns = []
    for i in range(number_of_turns):
        columns.extend([
            f"text_{i*2+1}", f"text_{i*2+2}",
            f"proposal_amount_round_{i+1}", f"proposal_acceptance_round_{i+1}"
        ])
    df = pd.DataFrame([dialogue_data], columns=columns)
    return dialogue_results, rounds_data, df

def encode_image(image_path):
    """
    Encode an image to base64 format.
    :param image_path: Path to the image
    :return: Base64 encoded image
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def save_progress(df_all: pd.DataFrame, base_path: str, start_folder_num: int, end_folder_num: int):
    """
    Save progress to an Excel file.
    :param df_all: The complete DataFrame
    :param base_path: Path to save the file
    :param start_folder_num: Starting folder number
    :param end_folder_num: Ending folder number
    """
    if df_all.empty:
        print("No data to save.")
        return

    columns_reordered = ['proposer', 'responder']
    number_of_turns = 4
    columns_text = []
    columns_proposals = []
    columns_acceptance = []
    for i in range(1, number_of_turns + 1):
        columns_text.extend([f"text_{i*2-1}", f"text_{i*2}"])
        columns_proposals.append(f"proposal_amount_round_{i}")
        columns_acceptance.append(f"proposal_acceptance_round_{i}")

    columns_reordered.extend(columns_text + columns_proposals + columns_acceptance)
    df_all = df_all.reindex(columns=columns_reordered, fill_value='')

    excel_file_path = os.path.join(base_path, f'study2_gpt_{start_folder_num}-{end_folder_num}.xlsx')
    df_all.to_excel(excel_file_path, index=False)
    print(f"DataFrame saved to {excel_file_path}")

# Prompts for both models
system_prompt1 = "In the following conversation, you are one of the two players participating in a negotiation game."
system_prompt2 = "In the following conversation, you are one of the two players participating in a negotiation game."

# Base path for data (replace with your own path)
base_path = "/path/to/your/folder"
subfolders = sorted([f for f in os.listdir(base_path) if not f.endswith('.png')])

# User input for folder processing range
start_folder_num = int(input("Enter the starting folder number (numeric only): "))
end_folder_num = int(input("Enter the ending folder number (numeric only): "))

df_all = pd.DataFrame()
skipped_files = []

print(f"Processing folders from {start_folder_num} to {end_folder_num}.")

# All folders
all_images = []
for i in range(1, 8):
    folder_path = os.path.join(base_path, str(i))
    images = sorted([f for f in os.listdir(folder_path) if f.endswith('.png')])
    all_images.append((folder_path, images))

# Selected folders
selected_folders = [(os.path.join(base_path, str(i)), sorted([f for f in os.listdir(os.path.join(base_path, str(i))) if f.endswith('.png')])) for i in range(start_folder_num, end_folder_num + 1)]

total_combinations = sum(len(images1) * len(images2) for (_, images1) in selected_folders for (_, images2) in all_images)
print(f"Total image pairs to process: {total_combinations}")

try:
    with tqdm(total=total_combinations, desc="Processing image pairs") as pbar:
        for folder1, images1 in selected_folders:
            for img1 in images1:
                for folder2, images2 in all_images:
                    for img2 in images2:
                        error_count = 0

                        while error_count < 5:
                            try:
                                image_path1 = os.path.join(folder1, img1)
                                image_path2 = os.path.join(folder2, img2)

                                base64_image1 = encode_image(image_path1)
                                base64_image2 = encode_image(image_path2)

                                img1_num = int(re.findall(r'\d+', img1)[0])
                                img2_num = int(re.findall(r'\d+', img2)[0])

                                user_prompt_model1 = [
                                    {"type": "text", "text": "You are the character in the following image:"},
                                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image1}"}},
                                    {"type": "text", "text": "The next image is your opponent:"},
                                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image2}"}},
                                    {"type": "text", "text": """
                                    Consider your character's traits in upcoming conversations.

                                    Let's play an ultimatum game. The rules of the game are as follows:
                                    (1) Two players divide $100 in each round. 
                                    (2) In each round, one player proposes a division ratio, and the other player can only accept or reject the proposal; one can not make a counter-proposal. 
                                    (3) If the proposal is accepted, the money is divided according to the proposed ratio; if rejected, neither player gets any money. 
                                    (4) The game consists of a total of four rounds, with players alternating roles between proposing and responding in each round. 

                                    You are the proposer in the first and third round.

                                    Do not declare '===round#===' on your own.

                                    The proposer should wait the '===round#===' notice and then start making the proposal.
                                    """}
                                ]
                                user_prompt_model2 = [
                                    {"type": "text", "text": "You are the character in the following image:"},
                                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image2}"}},
                                    {"type": "text", "text": "The next image is your opponent:"},
                                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image1}"}},
                                    {"type": "text", "text": """
                                    Consider your character's traits in upcoming conversations.

                                    Let's play an ultimatum game. The rules of the ultimatum game are as follows: 
                                    (1) Two players divide $100 in each round. 
                                    (2) In each round, one player proposes a division ratio, and the other player can only accept or reject the proposal. 
                                    (3) If the proposal is accepted, the money is divided according to the proposed ratio; if rejected, neither player gets any money. 
                                    (4) The game consists of a total of four rounds, with players alternating roles between proposing and accepting in each round. 

                                    You are the proposer in the second and fourth round.

                                    Do not declare '===round#===' on your own.

                                    The proposer should wait the '===round#===' notice and then start making the proposal.
                                    """}
                                ]

                                model1 = DialogModel()
                                model2 = DialogModel()

                                model1.set_prompts(system_prompt1, user_prompt_model1)
                                model2.set_prompts(system_prompt2, user_prompt_model2)
                                number_of_turns = 4
                                result, rounds_results, df = automated_dialogue(model1, model2, number_of_turns)
                                if result:
                                    df['proposer'] = img1_num
                                    df['responder'] = img2_num
                                    df_all = pd.concat([df_all, df], ignore_index=True)
                                else:
                                    skipped_files.append(img2_num)
                                model1.reset_dialog()
                                break

                            except Exception as e:
                                print(f"An error occurred: {e}")
                                error_count += 1
                                if error_count >= 5:
                                    print(f"Skipping file pair: {img1} and {img2} after 5 errors.")
                                    skipped_files.append((img1_num, img2_num))
                                    break
                        pbar.update(1)

except Exception as e:
    print(f"An error occurred: {e}")
    save_progress(df_all, base_path, start_folder_num, end_folder_num)
    print(f"Skipped files due to errors: {', '.join(map(str, skipped_files))}")
finally:
    save_progress(df_all, base_path, start_folder_num, end_folder_num)
    if skipped_files:
        print(f"Skipped files: {', '.join(map(str, skipped_files))}")


Processing folders from 1 to 7.
Total image pairs to process: 1225


Processing image pairs: 100%|██████████| 1225/1225 [16:31:15<00:00, 48.55s/it]  


DataFrame saved to C:/Users/dssalpc/Desktop/aai/study2_data\study2_gpt_1-7.xlsx
